# CSE 151B: Homework 2 Coding
## PyTorch Implementation

Using PyTorch’s `Sequential` model class, build a deep convolutional network to classify handwritten digits in MNIST.

You are only allowed to use the following in your model design:
- Linear Layers
- Conv2D
- MaxPool2D
- BatchNorm2D
- Dropout Layers
- ReLU and Softmax
- Flatten

Your goal is to build a model that achieves **test accuracy ≥ 0.985** with fewer than 1 million parameters.

**Warning**: The modules in your Sequential network should *only* consist of `nn` objects! That means you should not be using `torch.nn.functional` modules or lambda expressions in your Sequential block. Leaving functional/lambda expressions in your model code will result in no credit!

This notebook provides a skeleton layout for you. You may use whatever parts of this notebook you deem necessary; there is no need for you to adhere to the structure. However, during submission, you must carefully follow the zip file formatting as requested; see the bottom of the notebook.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
def get_data_loaders(batch_size) -> tuple[DataLoader, DataLoader]:
    '''
    Return the training and testing MNIST dataloaders.
    '''
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    # ========================
    # TODO: create dataloaders
    # ========================
    return ...


In [ ]:
def build_model(dropout_prob=0.5) -> nn.Module:
    model = nn.Sequential(
        # ==========================
        # TODO: fill in architecture
        # ==========================
    )
    return model


In [ ]:
def check_params():
    model = build_model()
    print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")

In [ ]:
def train(model, optimizer, criterion, train_loader, n_epochs = 1):
    '''
    Train the model for `n_epochs` epochs. Returns none (model is modified in place)
    '''
    model.train()
    # =====================
    # TODO: train the model
    # =====================

In [ ]:
def test(model, test_loader):
    '''
    Tests the model. Returns none (you should print the accuracy)
    '''
    model.eval()
    # =================================
    # TODO: evaluate the model accuracy
    # =================================

In [ ]:
# try 10 different dropout values
train_loader, test_loader = get_data_loaders()
criterion = ... # TODO: use a criterion/loss function
dropout_values = [i / 10 for i in range(10)]

for p in dropout_values:
    model = build_model(dropout_prob=p)
    optimizer = ... # TODO: use an optimizer
    train(model, optimizer, criterion, train_loader)
    test(model, test_loader)
    torch.save(model, f'hw2_dropout_{p}.pt')

In [ ]:
# find your best model, and train it for 10 epochs
best_p = ... # TODO: fill in your best probability
model = build_model(dropout_prob=best_p)
optimizer = ... # TODO: use an optimizer
train(model, optimizer, criterion, train_loader, n_epochs = 10)
test(model, test_loader)
torch.save(model, "hw2_model.pt")

# Submission Instructions

Zip all of your **code** and **model .pt files** into one file, and submit on Gradescope to the respective submission.